# 程式作業1

In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
#parameter1
s0 = 50
K1 = 50
K2 = 98
K3 = 102
K4 = 110
T = 1
r = 0.1
q = 0.05
sigma = 0.25

In [3]:
def v(K):
    return (np.log(s0/K)+(r-q + sigma**2 /2)*T ) / (sigma*np.sqrt(T))
def d(K):
    return (np.log(s0/K)+(r-q - sigma**2 /2)*T ) / (sigma*np.sqrt(T))
def fr(a,b):
    return norm.cdf(v(a)) - norm.cdf(v(b))
def fq(a,b):
    return norm.cdf(d(a)) - norm.cdf(d(b))

In [4]:
def blackprice(K): 
    return s0*np.exp(-q*T)*norm.cdf(v(K)) - np.exp(-r*T)*K*norm.cdf(d(K))
# put-call parity
def putprice(K):
    return blackprice(K) - s0*np.exp(-q*T) + K*np.exp(-r*T)

In [5]:
price = s0*np.exp(-q*T)*fr(K1,K2) - K1*np.exp(-r*T)*fq(K1,K2) +(K2-K1)*np.exp(-r*T)*fq(K2,K3) \
        +(K2-K1)/(K4-K3) *( K4*np.exp(-r*T)*fq(K3,K4) - s0*np.exp(-q*T)*fr(K3,K4) )
price

5.7576835603194185

# Black-Shcoles公式驗證:
* 用4個call來驗證數值

In [6]:
blackprice(K1)-blackprice(K2)-blackprice(K3)*(K2-K1)/(K4-K3)+blackprice(K4)*(K2-K1)/(K4-K3)

5.757683560319414

# 未來報酬公式
## 蒙地卡羅法
* 每次模擬含10000個股價
* 共模擬21次


In [7]:
def futurepay(s):
    if K1<=s<K2:
        return s-K1
    elif K2<=s<K3:
        return K2-K1
    elif K3<=s<K4:
        return (K2-K1)/(K4-K3) *(K4-s)
    else:
        return 0

In [8]:
a = []
for i in range(1,21):
    lnst = np.random.normal(loc=np.log(s0)+(r-q - sigma**2 /2)*T, scale=sigma*np.sqrt(T), size=10000)
    st = np.exp(lnst)
    payoff = [ np.exp(-r*T) * futurepay(i) for i in st]
    a.append(np.mean(payoff))  
mn = np.mean(a)
sd = np.std(a)
print ('95%%信賴區間為=(%f,%f)' %(mn-2*sd,mn+2*sd))
if mn-2*sd<=price<=mn+2*sd:
    print('%f為合理估計'%price)
else:
    print('估計錯誤')
    

95%信賴區間為=(5.601751,5.924040)
5.757684為合理估計


In [10]:
putprice(K1)

3.547582258350303